In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
import time
import keyboard

In [2]:
# df = pd.read_csv('csvfile.csv', header=None)

# # x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,1:].values,df.iloc[:,0].values)

# x_train = df.iloc[:,1:].values
# y_train = df.iloc[:,0].values

# x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
# y_train = np.reshape(y_train, (y_train.shape[0],1))
# y_train = tf.one_hot(y_train, depth=5)
# # x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))
# # y_test = np.reshape(y_test, (y_test.shape[0],1))
# # y_test = tf.one_hot(y_test, depth=3)

# def train_model():
#     model = tf.keras.Sequential([
#         tf.keras.layers.Dense(64, activation='relu', input_shape=(1, 63)),
#         tf.keras.layers.Dense(128, activation='relu'),
#         tf.keras.layers.Dense(128, activation='relu'),
#         tf.keras.layers.Dense(64, activation='relu'),
#         tf.keras.layers.Dense(32, activation='relu'),
#         tf.keras.layers.Dense(16, activation='relu'),
#         tf.keras.layers.Dense(5, activation='softmax')
#     ])


#     return model

# model = train_model()
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.fit(x_train, y_train, batch_size=32, epochs=20, validation_split=0.2)
# model.summary()
# model.save('hand_gest.h5')

In [2]:
gestures = ['open','closed','flip','thumbs-up','point']
model = tf.keras.models.load_model('hand_gest.h5')

In [3]:
num_of_points = 90
reg_time = 0.5
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
drawing_spec_landmark = mp_drawing.DrawingSpec(color=(0, 0, 255), circle_radius=1, thickness=5)
drawing_spec_line = mp_drawing.DrawingSpec(color=(0, 255, 0), circle_radius=5, thickness=2)

last_prediction_time = 0
circle_centers = []
y_pred = np.array([])
# For webcam input:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)
    image_height, image_width, _ = image.shape
    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            drawing_spec_landmark,drawing_spec_line)
        

    if keyboard.is_pressed('q'): 
      a = results.multi_hand_landmarks
      for i in range(len(a)):
        landmarks_list = []
        for id, lm in enumerate(a[i].landmark):
          landmarks_list.append(lm.x)
          landmarks_list.append(lm.y)
          landmarks_list.append(lm.z)
        normalized_landmarks = [((x - min(landmarks_list)) / (max(landmarks_list) - min(landmarks_list))) for x in landmarks_list]
        normalized_landmarks = np.insert(normalized_landmarks,0,4)
        with open('csvfile.csv','a') as file:
          np.savetxt(file,normalized_landmarks.reshape(1,-1),delimiter=',')
    
    current_time = time.time()
    if current_time - last_prediction_time >= reg_time:
      last_prediction_time = current_time
      if results.multi_hand_landmarks:
        a = results.multi_hand_landmarks
        for i in range(len(a)):
          landmarks_list = []
          for id, lm in enumerate(a[i].landmark):
            landmarks_list.append(lm.x)
            landmarks_list.append(lm.y)
            landmarks_list.append(lm.z)
          normalized_landmarks = [((x - min(landmarks_list)) / (max(landmarks_list) - min(landmarks_list))) for x in landmarks_list]
          pred = np.array(normalized_landmarks)
          pred = np.reshape(pred, (1, 1, pred.shape[0]))
          y_pred = model.predict(pred)
          # print(gestures[np.argmax(y_pred[0])])
        if (np.argmax(y_pred[0]) == 4):
          x = landmarks_list[24] * image_width
          y = landmarks_list[25] * image_height
          circle_centers.append((int(x), int(y)))
        if (np.argmax(y_pred[0]) == 2):
          circle_centers = []

    for center in circle_centers:
      cv2.circle(img = image, center = center, radius = 2, color = (0, 0, 255), thickness =5)
    if len(circle_centers) > num_of_points:
        circle_centers.pop(0)

    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))

    if keyboard.is_pressed('s'):
      break
    if cv2.waitKey(5) & 0xFF == 27:
      break


cap.release()
cv2.destroyAllWindows()